This is an attempt to recreate the parameter estimation [example](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/fig-html/appendix/fig-A-10.html) from James Rawlings book on [Reactor Design](https://sites.engineering.ucsb.edu/~jbraw/chemreacfun/) using Pyomo [parmest](https://pyomo.readthedocs.io/en/stable/contributed_packages/parmest/driver.html).

In [1]:
# Import libraries
from pyomo.environ import *
from pyomo.dae import *
import pyomo.contrib.parmest.parmest as parmest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from pyomo.contrib.interior_point.interface import InteriorPointInterface

This example has a series reaction $A \rightarrow B \rightarrow C$. The dataset consists of measures concentrations of A, B and C over time. The goal is to estimate the rate constants $k_1$ and $k_2$ for the two reactions.

In [3]:
data_df = pd.read_csv("ABC_data.csv")
data_df.head()

,t,ca,cb,cc
0,0.000,0.957,-0.031,-0.015
1,0.263,0.557,0.330,0.044
2,0.526,0.342,0.512,0.156
3,0.789,0.224,0.499,0.310
4,1.053,0.123,0.428,0.454


In [4]:
# Convert data to a list of dictionaries
data = [{'ca_meas': {k:v for (k, v) in zip(data_df.t, data_df.ca)},
    'cb_meas': {k:v for (k, v) in zip(data_df.t, data_df.cb)},
    'cc_meas': {k:v for (k, v) in zip(data_df.t, data_df.cc)} }]

In [5]:
#
# Define the model 
#
def ABC_model(data):
    
    ca_meas = data['ca_meas']
    cb_meas = data['cb_meas']
    cc_meas = data['cc_meas']
    
    meas_t = list(ca_meas.keys())
       
    ca0 = 1.0
    cb0 = 0.0
    cc0 = 0.0
        
    m = ConcreteModel()
    
    #m.k1 = Var(initialize = 0.5, bounds = (1e-4, 10))
    #m.k2 = Var(initialize = 3.0, bounds = (1e-4, 10))
    m.k1 = Var(initialize = 2.0)
    m.k2 = Var(initialize = 1.0)
    
    m.time = ContinuousSet(bounds = (0.0, 5.0), initialize = meas_t)
    #m.ca = Var(m.time, initialize = ca0, bounds = (0, ca0))
    #m.cb = Var(m.time, initialize = cb0, bounds = (0, ca0))
    #m.cc = Var(m.time, initialize = cc0, bounds = (0, ca0))
    
    m.ca = Var(m.time, initialize = ca0)
    m.cb = Var(m.time, initialize = cb0)
    m.cc = Var(m.time, initialize = cc0)
    
    m.dca = DerivativeVar(m.ca, wrt = m.time)
    m.dcb = DerivativeVar(m.cb, wrt = m.time)
    m.dcc = DerivativeVar(m.cc, wrt = m.time)
    
    def _dcarate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dca[t] == -m.k1 * m.ca[t]
    m.dcarate = Constraint(m.time, rule = _dcarate)
    
    def _dcbrate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dcb[t] == m.k1 * m.ca[t] - m.k2 * m.cb[t]
    m.dcbrate = Constraint(m.time, rule = _dcbrate)
    
    def _dccrate(m, t):
        if t == 0:
            return Constraint.Skip
        else:
            return m.dcc[t] == m.k2 * m.cb[t]
    m.dccrate = Constraint(m.time, rule = _dccrate)
    
    def _initcon(m):
        yield m.ca[m.time.first()] == ca0
        yield m.cb[m.time.first()] == cb0
        yield m.cc[m.time.first()] == cc0
    m.initcon = ConstraintList(rule = _initcon)
    
    def ComputeFirstStageCost_rule(m):
        return 0
    m.FirstStageCost = Expression(rule=ComputeFirstStageCost_rule)

    def ComputeSecondStageCost_rule(m):
        return sum((m.ca[t] - ca_meas[t]) ** 2 + (m.cb[t] - cb_meas[t]) ** 2 
                   + (m.cc[t] - cc_meas[t]) ** 2 for t in meas_t) 
    m.SecondStageCost = Expression(rule=ComputeSecondStageCost_rule)

    
    def total_cost_rule(model):
        return model.FirstStageCost + model.SecondStageCost
    m.Total_Cost_Objective = Objective(rule=total_cost_rule, sense=minimize)
    
    disc = TransformationFactory('dae.collocation')
    disc.apply_to(m, nfe=20, ncp=2)
    
    return m

In [6]:
# parameters to be estimated
theta_names = ['k1', 'k2']

In [7]:
# First tried collocation method. Max iterations are exceeded
pest = parmest.Estimator(ABC_model, data, theta_names, tee = True)
res = pest.theta_est()

Ipopt 3.12: 
Exception of type: OPTION_INVALID in file "../../../../Ipopt/src/Common/IpOptionsList.cpp" at line 639:
 Exception message: Read Option: "compute_red_hessian". It is not a valid option. Check the list of available options.
ampl_ipopt.cpp: Error in second Initialize!!!!
ERROR: Solver (ipopt) returned non-zero return code (155)
ERROR: See the solver log above for diagnostic information.


ApplicationError: Solver (ipopt) did not exit normally

In [ ]:
m = ABC_model(data[0])

In [ ]:
if not hasattr(m, 'ipopt_zL_out'):
    m.ipopt_zL_out = Suffix(direction=Suffix.IMPORT)
if not hasattr(m, 'ipopt_zU_out'):
    m.ipopt_zU_out = Suffix(direction=Suffix.IMPORT)
if not hasattr(m, 'ipopt_zL_in'):
    m.ipopt_zL_in = Suffix(direction=Suffix.EXPORT)
if not hasattr(m, 'ipopt_zU_in'):
    m.ipopt_zU_in = Suffix(direction=Suffix.EXPORT)
if not hasattr(m, 'dual'):
    m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

In [ ]:
solver = SolverFactory('ipopt')
solver.options['bound_relax_factor']=0
solver.options['honor_original_bounds']='no'

In [ ]:
status = solver.solve(m, tee=True)

In [ ]:
estimated_mu = list()
for v in m.ipopt_zL_out:
    if v.has_lb():
        estimated_mu.append((value(v) - v.lb)*m.ipopt_zL_out[v])
for v in m.ipopt_zU_out:
    if v.has_ub():
        estimated_mu.append((v.ub - value(v))*m.ipopt_zU_out[v])
if len(estimated_mu) == 0:
    mu = 10**-8.6
else:
    mu = sum(estimated_mu)/len(estimated_mu)
    # check to make sure these estimates were all reasonable
    if any([abs(mu-estmu) > 1e-7 for estmu in estimated_mu]):
        print('Warning: estimated values of mu do not seem consistent - using mu=10^(-8.6)')
        mu = 10**-8.6

In [ ]:

for i, v in enumerate(m.ipopt_zL_out):
    print(v, v.lb, v.ub, value(v), m.ipopt_zL_out[v], m.ipopt_zU_out[v], estimated_mu[i])

In [ ]:
any([abs(mu-estmu) > 1e-7 for estmu in estimated_mu])

In [ ]:
independent_variables = [m.k1, m.k2]
ind_vardatas = list()
for v in independent_variables:
    if v.is_indexed():
        for k in v:
            ind_vardatas.append(v[k])
    else:
        ind_vardatas.append(v)

In [ ]:
for v in ind_vardatas:
    if (v.has_lb() and pyo.value(v) - v.lb <= bound_tolerance) or \
       (v.has_ub() and v.ub - pyo.value(b) <= bound_tolerance):
            raise ValueError("Independent variable: {} has a solution value that is near"
                             " its bound (according to tolerance). The reduced hessian"
                             " computation does not support this at this time. All"
                             " independent variables should be in their interior.".format(v))

In [ ]:
kkt_builder = InteriorPointInterface(m)

In [ ]:
pyomo_nlp = kkt_builder.pyomo_nlp()

In [ ]:
ind_var_indices = pyomo_nlp.get_primal_indices(ind_vardatas)

In [ ]:
kkt_builder.set_barrier_parameter(mu)
kkt = kkt_builder.evaluate_primal_dual_kkt_matrix()

In [ ]:
primals = kkt_builder._nlp.get_primals()

In [ ]:
duals_primals_lb = kkt_builder._duals_primals_lb
duals_primals_ub = kkt_builder._duals_primals_ub

In [ ]:
#duals_primals_lb / (primals - kkt_builder._nlp.primals_lb())
duals_primals_ub / (kkt_builder._nlp.primals_ub() - primals)

In [ ]:
duals_primals_ub[82], kkt_builder._nlp.primals_ub()[82], primals[82]